In [1]:
# load packages
import os
import sys
import re
import random
import numpy as np
import pandas as pd
import tensorflow as tf
tf.__version__

'1.12.0'

## Prepare Data

In [2]:
data = pd.read_csv("data/iris-dataset.csv") # load data
type(data)

pandas.core.frame.DataFrame

In [3]:
data.shape

(150, 6)

In [4]:
data.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
target_values = data.Species.unique()
target_values

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [6]:
# create mapping string -> numeric value
target_values_mapping = dict()
i = 0
for tar in target_values:
    if tar not in target_values_mapping:
        target_values_mapping[tar] = i
        i += 1
    else:
        continue

In [7]:
target_values_mapping

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

In [8]:
# convert string value to numeric values
data.Species = data.Species.apply(lambda x: target_values_mapping[x])
data.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


### Selecting Only 2 Classes -> Converting To Binary Classification

In [9]:
data = data[data.Species <= 1]
data.shape

(100, 6)

In [10]:
data.Species.unique()

array([0, 1])

In [11]:
features = np.array(data.iloc[:, 1:-1]) # load features into a numpy array
features.shape

(100, 4)

In [12]:
labels = np.array(data.iloc[:, [-1]]) # load target values into a numpy array
labels.shape

(100, 1)

In [13]:
features[0], labels[0]

(array([5.1, 3.5, 1.4, 0.2]), array([0]))

In [14]:
features[1], labels[1]

(array([4.9, 3. , 1.4, 0.2]), array([0]))

In [15]:
# split data into train and validate
val_split_size = 0.2
random_indices = np.random.rand(len(features)) <= val_split_size

# split data
train_x = features[~random_indices]
val_x = features[random_indices]
train_y = labels[~random_indices]
val_y = labels[random_indices]

In [16]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape

((88, 4), (88, 1), (12, 4), (12, 1))

# Vectorized Logistic Regression Model

In [17]:
# placeholders
x = tf.placeholder(dtype=tf.float32, shape=(None, features.shape[1]), name="feature_x")
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name="target_y")

In [18]:
# define weights and bias
w = tf.Variable(tf.random_normal(shape=(1, features.shape[1]), dtype=tf.float32, mean=0.0, stddev=0.1))
b = tf.Variable(tf.zeros(shape=(1, 1), dtype=tf.float32))

In [19]:
logistic_model = tf.sigmoid(tf.add(tf.matmul(x, tf.transpose(w)), b))

In [20]:
# loss to minimize
loss_function = tf.reduce_mean(-(y*tf.log(logistic_model) + (1 - y)*tf.log(1 - logistic_model))) # binary cross entropy

## Using Inbuilt Optimizer

In [21]:
trainer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss_function)

In [22]:
# train the model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("[Training...]")
    for epoch in range(10):
        sess.run(trainer, feed_dict={x:train_x, y:train_y})
        train_loss = sess.run(loss_function, feed_dict={x:train_x, y:train_y})
        print("Step:", epoch, "\tTrain Loss:", train_loss, end="\t")
        val_loss = sess.run(loss_function, feed_dict={x:val_x, y:val_y})
        print("Val Loss:", val_loss)
    print("Final Weights:", sess.run(w))
    predicted_y = sess.run(logistic_model, feed_dict={x:val_x, y:val_y})

[Training...]
Step: 0 	Train Loss: 0.5430743	Val Loss: 0.5674382
Step: 1 	Train Loss: 0.47200394	Val Loss: 0.4925133
Step: 2 	Train Loss: 0.3943813	Val Loss: 0.39631525
Step: 3 	Train Loss: 0.33927426	Val Loss: 0.3242364
Step: 4 	Train Loss: 0.29528952	Val Loss: 0.27345732
Step: 5 	Train Loss: 0.25094116	Val Loss: 0.23194475
Step: 6 	Train Loss: 0.21222673	Val Loss: 0.19988249
Step: 7 	Train Loss: 0.18310398	Val Loss: 0.1765037
Step: 8 	Train Loss: 0.1603501	Val Loss: 0.15669073
Step: 9 	Train Loss: 0.14019422	Val Loss: 0.13682549
Final Weights: [[ 0.11733446 -0.97399986  0.7963999   1.0043694 ]]


### > Prediction Analysis

In [23]:
# clip the prediction on threshold of 0.5 to get final prediction
final_prediction = list()
for pred in predicted_y:
    if pred > 0.5:
        final_prediction.append(1)
    else:
        final_prediction.append(0)
final_prediction = np.array(final_prediction).reshape((len(final_prediction), 1))
final_prediction.shape, val_y.shape

((12, 1), (12, 1))

In [24]:
# find accuracy
correct_prediciton = 0
for i in range(val_y.shape[0]):
    if final_prediction[i] == val_y[i]:
        correct_prediciton += 1
    else:
        continue
"Accuracy = ", correct_prediciton / val_y.shape[0] * 100

('Accuracy = ', 100.0)

## Use Your Own Gradient Descent

In [25]:
# compute gradients
grad_w, grad_b = tf.gradients(xs=[w, b], ys=loss_function)
# update
new_w = w.assign(w - 0.1*grad_w)
new_b = b.assign(b - 0.1*grad_b)

In [26]:
# train the model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("[Training...]")
    for epoch in range(10):
        _, _, train_loss = sess.run([new_w, new_b, loss_function], feed_dict={x:train_x, y:train_y})
        print("Step:", epoch, "\tTrain Loss:", train_loss, end="\t")
        val_loss = sess.run(loss_function, feed_dict={x:val_x, y:val_y})
        print("Val Loss:", val_loss)
    print("Final Weights:", sess.run(w))
    predicted_y = sess.run(logistic_model, feed_dict={x:val_x, y:val_y})

[Training...]
Step: 0 	Train Loss: 0.6490481	Val Loss: 0.62043035
Step: 1 	Train Loss: 0.6003631	Val Loss: 0.5768913
Step: 2 	Train Loss: 0.56400037	Val Loss: 0.5441076
Step: 3 	Train Loss: 0.53140634	Val Loss: 0.5129469
Step: 4 	Train Loss: 0.50171995	Val Loss: 0.48477176
Step: 5 	Train Loss: 0.47462568	Val Loss: 0.4589727
Step: 6 	Train Loss: 0.44985744	Val Loss: 0.43535772
Step: 7 	Train Loss: 0.42717648	Val Loss: 0.41369376
Step: 8 	Train Loss: 0.40636876	Val Loss: 0.39378253
Step: 9 	Train Loss: 0.387243	Val Loss: 0.3754464
Final Weights: [[-0.05141738 -0.2923927   0.40628156  0.346352  ]]


### > Prediction Analysis

In [27]:
# clip the prediction on threshold of 0.5 to get final prediction
final_prediction = list()
for pred in predicted_y:
    if pred > 0.5:
        final_prediction.append(1)
    else:
        final_prediction.append(0)
final_prediction = np.array(final_prediction).reshape((len(final_prediction), 1))
final_prediction.shape, val_y.shape

((12, 1), (12, 1))

In [28]:
# find accuracy
correct_prediciton = 0
for i in range(val_y.shape[0]):
    if final_prediction[i] == val_y[i]:
        correct_prediciton += 1
    else:
        continue
"Accuracy = ", correct_prediciton / val_y.shape[0] * 100

('Accuracy = ', 100.0)

Perfect Score!